In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from ngramsapi import get_word_trends
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [15]:
tools_list = [{
  "type": "function",
  "function": {
    "name": "get_word_trends",
    "description": "Get the trends of a word over time using the google ngrams api",
    "parameters": {
      "type": "object",
      "properties": {
        "query": {
          "type": "string",
          "description": "The word to get the trends for"
        },
        "start_year": {
          "type": "integer",
          "description": "The start year for the trends"
        },
        "end_year": {
          "type": "integer",
          "description": "The end year for the trends"
        },
        "corpus": {
          "type": "string",
          "description": "The corpus to use for the trends"
        },
        "smoothing": {
          "type": "integer",
          "description": "The smoothing factor for the trends"
        },
        "case_insensitive": {
          "type": "boolean",
          "description": "Flag to indicate if the search should be case insensitive"
        },
      },
      "required": ["query"]
    }
    
  }
}]

In [16]:
assistant = client.beta.assistants.retrieve(assistant_id=os.getenv("OPENAI_ASSISTANT_ID"))

In [17]:
# assistant
client.beta.assistants.update(
  assistant_id=assistant.id,
  tools=tools_list
)

Assistant(id='asst_NPJ7zjXnm6ShHHw5RuCldZDF', created_at=1720033291, description=None, instructions='EVERY answer should start with the: trial ID, uri, year and date in list format:\n- TrialAccount id\n- collection\n- uri\n- year\n- date\n\n1. Understanding the Old Bailey Database:\nContext: The Old Bailey database contains records of criminal trials held at the Central Criminal Court in London, commonly known as the Old Bailey, from 1674 to 1913.\nContent: The database includes detailed accounts of the proceedings, including the names of the accused, the charges, the verdicts, and the sentences.\n\n2. Query Structure:\nSpecificity: Encourage users to be as specific as possible in their queries to facilitate efficient searches (e.g., specific dates, names, types of crime).\nExamples:\n"Find all trials involving theft in 1782."\n"What was the verdict in the trial of John Doe in 1805?"\n"List all female defendants tried for murder between 1800 and 1850."\n\n\n3. Database Navigation:\nSea

In [22]:
from typing_extensions import override
from openai import AssistantEventHandler

arguments = []
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == "function":
      print(delta.function)

In [30]:
threadId = ""

message = client.beta.threads.messages.create(
  thread_id=threadId,
  role="user",
  content="Can you show me the trends for the word 'openai' from 2010 to 2020?"
)

run = client.beta.threads.runs.create_and_poll(
  thread_id=threadId,
  assistant_id=assistant.id,
)

if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=threadId
  )
  print(messages)
else:
  print(run.status)
  
tool_outputs = []
for tool in run.required_action.submit_tool_outputs.tool_calls:
  if tool.function.name == "get_current_temperature":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": "57"
    })
  elif tool.function.name == "get_rain_probability":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": "0.06"
    })

# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=threadId,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")
 
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=threadId
  )
  print(messages)
else:
  print(run.status)

requires_action
No tool outputs to submit.
requires_action
